In [ ]:
import os 
import re
import csv 
from glob import glob
import random
from tqdm import tqdm
import numpy as np
import anndata
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sp
import squidpy as sq
from collections import defaultdict,Counter
# from pysam import AlignmentFile

import scipy.stats as st
from scipy import sparse, stats
import gzip
import json

In [ ]:
#### compara analysis among three samples from ST-FFPE-mIF

In [ ]:
def getOffset(tissueGem):
    offsetX = 0
    offsetY = 0
    reader = gzip.open(tissueGem, 'rt') if tissueGem.endswith(".gz") else open(tissueGem, 'r')
    for line in reader:
        if not line.startswith("#"):
            break
        elif "OffsetX" in line:
            offsetX = int(line.strip().split("=")[1])
        elif "OffsetY" in line:
            offsetY = int(line.strip().split("=")[1])
    reader.close()
    return offsetX, offsetY

def _read_gem(gemf):

    expColumnTypes = {"x": np.uint32, "y": np.uint32, "geneID": 'category', "geneName": 'category', "MIDCount": np.uint32, "UMI": np.uint64, "reads": np.uint16}
    gem = pd.read_csv(gemf, sep="\t", comment="#", dtype=expColumnTypes, quoting=csv.QUOTE_NONE)
    offsetX, offsetY = getOffset(gemf)
    gem['x'] = gem['x'] + offsetX
    gem['y'] = gem['y'] + offsetY

    if "geneName" in gem.columns:
        gem = gem[['geneName','x','y','MIDCount']]
        gem.columns = ['geneID','x','y','MIDCount']
    else:
        gem = gem[['geneID','x','y','MIDCount']]
    return gem

def gem2h5ad(geneExpFile, binSize=50, tissueMask=None):
    print (f"bin size has been set to {binSize}")
    columntypes = {"geneID": 'category',"geneName":"category", "x": int, 'y': int, "UMICount": int, "MIDCount": int, "MIDCounts": int, "values": int}
    df = pd.read_csv(geneExpFile, sep="\t", quoting=csv.QUOTE_NONE, comment="#", dtype=columntypes)
    if "MIDCounts" in df.columns:
        df.rename(columns={"MIDCounts": "UMICount"}, inplace=True)
    elif 'values' in df.columns:
        df.rename(columns={"values": "UMICount"}, inplace=True)
    elif 'MIDCount' in df.columns:
        df.rename(columns={'MIDCount': 'UMICount'}, inplace=True)
    
    if "geneName" in df.columns:
        df = df[["x","y","geneName","UMICount"]]
        df.rename(columns={'geneName': 'geneID'}, inplace=True)
        
    if tissueMask != None:
        offsetX, offsetY = getOffset(geneExpFile)
        df['x'] = df['x'] + offsetX
        df['y'] = df['y'] + offsetY
        mask = tifffile.imread(tissueMask)
        maxY, maxX = mask.shape
        if maxX > df['x'].max() or maxY > df['y'].max():
            print ("WARMING: mask is out of bounds")
            df = df.loc[(df['x'] < maxX)&(df['y'] < maxY)]
        df = df.loc[mask[df['y'], df['x']] > 0]
    
    halfBinSize = int(binSize/2)
    
    #df['x'] = (df['x']/binSize).astype(int)*binSize + halfBinSize
    #df['y'] = (df['y']/binSize).astype(int)*binSize + halfBinSize 
    df['x'] = (df['x']//binSize)*binSize + halfBinSize
    df['y'] = (df['y']//binSize)*binSize + halfBinSize 
    df['cell'] = df['x'].astype(str) + "-" + df['y'].astype(str)

    cells = df['cell'].unique()
    genes = df['geneID'].unique()
    
    cells_dict = dict(zip(cells, range(0, len(cells))))
    genes_dict = dict(zip(genes, range(0, len(genes))))
    rows = df['cell'].map(cells_dict)
    cols = df['geneID'].map(genes_dict)
    expMtx = sparse.csr_matrix((df['UMICount'].values, (rows, cols)), shape=(cells.shape[0], genes.shape[0]), dtype=np.int32)

    obs = pd.DataFrame(index = cells)
    var = pd.DataFrame(index = genes)
    adata = anndata.AnnData(X = expMtx, obs = obs, var = var)
    positions = np.array(list(map(lambda x: [int(v) for v in x.strip().split("-")], adata.obs.index)))
    adata.obs['x'] = positions[:,0]
    adata.obs['y'] = positions[:,1]
    adata.obsm['spatial'] = adata.obs[['x', 'y']].values
    return adata

def readjson(jfile):
    with open(jfile,'r',encoding='utf8') as jf:
        json_data = json.loads(jf.read())
    return json_data

In [ ]:
from matplotlib.lines import Line2D
def violinplot(adata, groupby,key,setcolors,title,inner = None):
   
    matrix = adata.obs
    obs_tidy = pd.DataFrame(matrix)
    fig,axs = plt.subplots(1,1,figsize=(5,4))
    ax = sns.violinplot(x=groupby, y=key, data=obs_tidy,palette=setcolors,inner= inner)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_title(title,fontsize=14)
    ax.set_xticklabels(ax.get_xticklabels(), ha='center')
    return ax

def violinplot_allsamp(adata, groupby, key, setcolors,title,inner = None):
    
    setcolors = {k: v for k, v in setcolors.items() if k in set(adata.obs[groupby])}
    matrix = adata.obs
    obs_tidy = pd.DataFrame(matrix)
    fig,axs = plt.subplots(1,1,figsize=(5,4))
    ax = sns.violinplot(x=groupby, y=key, data=obs_tidy, palette=setcolors,inner= inner)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.spines['top'].set_visible(False)  
    ax.spines['right'].set_visible(False)
    ax.set_title(title,fontsize=14)
    ax.set_xticklabels(ax.get_xticklabels(), ha='right',rotation=50)
    custom_lines = [Line2D([0], [0], marker='s', color='w', markerfacecolor=color, markersize=10, label=k) for k,color in setcolors.items()]
    plt.legend(handles=custom_lines, title='', loc='upper left', bbox_to_anchor=(1, 1))

    return ax




def _boxplot(adata, groupby,key,setcolors,title):
    
    matrix = adata.obs
    obs_tidy = pd.DataFrame(matrix)
    fig,axs = plt.subplots(1,1,figsize=(4,4))
    ax = sns.boxplot(x=groupby, y=key, data=obs_tidy, palette=setcolors,width=0.7,showfliers = False)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.spines['top'].set_visible(False)  
    ax.spines['right'].set_visible(False)  
    ax.set_title(title,fontsize=14)
    return ax

In [ ]:
group = {"ST-FFPE-mIF":['Y00951M4', 'Y00951M3', 'Y00951M2'],
         "Stereo-seq-FFPE":['Y00951M1','Y00951M5',"D04699E3"]}

labels = {'Y00951M1':"Stereo-seq-FFPE_1",'Y00951M5':"Stereo-seq-FFPE_2","D04699E3":"Stereo-seq-FFPE_3",
          'Y00951M2':"ST-FFPE-mIF_1", 'Y00951M3':"ST-FFPE-mIF_2", 'Y00951M4':"ST-FFPE-mIF_3"
         }
label_group = {labels[s]: k for k,v in group.items() for s in v}

# colors
colors = {'Y00951M1':"#55B7E6", 'Y00951M2':"#193E8F", 'Y00951M3':"#E53528", 'Y00951M4':"#F09739", 'Y00951M5':"#C59D94","D04699E3":"#BAD65D"}
groupcolors = {s:cor for s,cor in zip(group.keys(),["#4DBAD6","#E44A33"])}
labcolors = {labels[s]:colors[s] for s in labels}

samples = pd.DataFrame({"sample": labels.keys(),
                      "lable":labels.values(),
                      "group":[label_group[x] for x in labels.values()],
                      "color_lab":[colors[x] for x in labels.keys()],
                      "color_group":[groupcolors[label_group[x]] for x in labels.values()]

In [ ]:
dlist = []
for samp in samps:
    adata = gem2h5ad(f"../00.data/Data_add/{samp}/{samp}.tissue.gem", binSize = binsize)
    dlist.append(adata)
adata_crc = sc.AnnData.concatenate(*dlist, join="outer", batch_key="sample",
                                  batch_categories=samps,
                                  fill_value=0)
adata_crc.var_names_make_unique()
adata_crc.var["mt"] = adata_crc.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata_crc, qc_vars=["mt"], inplace=True)

In [ ]:
violinplot_allsamp(adata_crc,groupby = "labels",key = "n_genes_by_counts",setcolors = labcolors,
           title = "nFeature_RNA")
violinplot_allsamp(adata_crc,groupby = "labels",key = "total_counts",setcolors = labcolors,title = "nCount_RNA")

In [ ]:
def exp_corplot(exp_raw,out_dir = None):

    cola,colb = exp_raw.columns
    exp_raw = exp_raw.applymap(lambda x:np.log2(x+1))
    
    pear_frame = exp_raw.corr()
    
    fig, axs = plt.subplots(figsize=(4.5, 4), constrained_layout=True)
    axs.scatter(list(exp_raw[cola]), list(exp_raw[colb]), color="black", s=24,alpha =0.1)
    slope, intercept, r_value, p_value, std_err = st.linregress(list(exp_raw[cola]),list(exp_raw[colb]))
    # txt = "Goodness of Fitting:R\u00b2={0:.2f}\nPearson Correlation:{1:.2f}\nPvalue:{2:.2e}".\
    #   format(r_value**2,pear_frame.loc[cola,colb],p_value)
    y_fit = slope * exp_raw[cola] + intercept
    # 绘制拟合线
    axs.plot(list(exp_raw[cola]), y_fit, color='#E53528', label='Fitted line')

    txt = "Goodness of Fitting:R\u00b2={0:.2f}\nPearson Correlation:{1:.2f}".format(r_value**2, pear_frame.loc[cola,colb])
    axs.text(0.05, 0.9,txt,ha="left", va="center", transform=axs.transAxes)
    plt.xlabel(f"{cola} gene expression", fontsize=12)
    plt.ylabel(f"{colb} gene expression", fontsize=12)
    
    if out_dir is not None:
        plt.savefig(f"{out_dir}/Pearson Correlation between {cola} and {colb}.png",bbox_inches='tight')

In [ ]:
def read_gems(samps):
    all_gems = []
    for samp in samps:
        bamfile = f"../00.data/Data_add/{samp}/{samp}_*/result/{samp}/outs/bam/annotated_bam/{samp}.Aligned.sortedByCoord.out.merge.q10.dedup.target.bam"
        gemfile = f"../00.data/Data_add/{samp}/{samp}.tissue.gem"
        bamfile = glob(bamfile)[0]
        print(gemfile)
        gem = _read_gem(gemfile)
        gene_bulkexp = gem.groupby(['geneID'], as_index=True)['MIDCount'].sum()
        all_gems.append(gene_bulkexp)
    genes_bulk = pd.concat(all_gems,axis=1)
    genes_bulk.columns = samps
    return genes_bulk

bulk_gems = read_gems(samps)
bulk_gems = bulk_gems.fillna(0)
bulk_gems.columns = [labels[x] for x in bulk_gems.columns]

In [ ]:
gtffile = "genomics/GRCh38_dna_primary_assembly_93_29/GRCh38_dna_primary_assembly_93/STAR/genes/genes.gtf"

gene_info = defaultdict(lambda :defaultdict(list))
with open(gtffile, 'r') as gtf_f: 

    for line in gtf_f:  

        if line.startswith('#'):    
            continue   

        fields = line.strip().split('\t')
        # if not fields[0].startswith("chr"):continue

        if fields[2] == "gene":
            gene = re.findall(r"gene_name \"(.*?)\";",fields[8])[0].strip()
            gene_type = re.findall(r"gene_biotype \"(.*?)\"",fields[8])[0].strip()

            gene_info[gene]= gene_type
exp_bulk = bulk_gems.copy()
exp_bulk_filter = exp_bulk[exp_bulk.index.map(lambda x:gene_info[x] not in ['lincRNA','antisense'])]
exp_bulk_filter = exp_bulk_filter[exp_bulk_filter.apply(lambda x: sum(x>2)>0,axis=1)]

In [ ]:
genetypes = Counter([gene_info[x] for x in exp_bulk.index ])
size =12
fig = plt.figure(figsize=(6,5),dpi=100)
ax = sns.barplot(x=genetypes.keys(),y=genetypes.values(),color="#EA8379")
ax.set_xlabel('',size =size)
ax.set_ylabel('MID Count',size =size+3)
plt.setp(ax.get_xticklabels(), rotation=45, ha='right',size =size-2)
sns.despine(top=True, right=True)

In [ ]:
for i in range(len(exp_bulk.columns)):
    for j in range(i+1,len(exp_bulk.columns)):
        exp_corplot(exp_bulk[[exp_bulk.columns[i],exp_bulk.columns[j]]],out_dir = "./figures/correlation/")

In [ ]:
exp_bulk_log = exp_bulk.applymap(lambda x:np.log2(x+1))
pear_frame = exp_bulk_log.corr()

pear_frame = pear_frame.sort_index()
sorted_columns = sorted(pear_frame.columns)
pear_frame = pear_frame.reindex(columns=sorted_columns)

fig, axs = plt.subplots(figsize=(5, 4), constrained_layout=True)
sns.heatmap(data=pear_frame, annot=True, 
            fmt=".2f",
            cmap='RdPu')
plt.title("Gene expression correlation",fontsize = 14)
plt.xticks(rotation=60,fontsize = 12,ha="center")
plt.yticks(rotation=0,fontsize = 12)

In [ ]:
#### calculate morna index

In [ ]:
# QC Filter cells
ann_dict = {}
for samp in samps:
    adatafile = f"./cache/adata_{samp}.h5ad"
    adata = sc.read_h5ad(adatafile)
    adata.var.index = adata.var.index.astype(str)
    adata.var_names_make_unique()
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
    
    sc.pp.filter_cells(adata, min_counts=50)
    sc.pp.filter_genes(adata, min_cells=10)
    adata = adata[adata.obs.pct_counts_mt < 10, :].copy()
    adata.raw = adata
    ann_dict[labels[samp]] = adata

In [ ]:
# Normalization
for v in ann_dict.values():
    sc.pp.normalize_total(v)
    sc.pp.log1p(v)
    sc.pp.scale(v, max_value=10)

In [ ]:
for v in ann_dict.values():
    # Moran's Score
    sq.gr.spatial_neighbors(v, coord_type="generic", spatial_key="spatial")
    sq.gr.spatial_autocorr(v, mode="moran")

In [ ]:
# Get First 10 moran variable genes, per sample
num_view = 10

top_autocorr = { 
    k: v.uns["moranI"]["I"].sort_values(ascending=False).head(num_view).index.tolist() 
    for k, v in ann_dict.items()
}

In [ ]:
ss = list(top_autocorr.keys())
top_autocorr_baised = set(top_autocorr[ss[0]]) & set(top_autocorr[ss[1]]) & set(top_autocorr[ss[2]]) &\
                        set(top_autocorr[ss[3]]) & set(top_autocorr[ss[4]]) & set(top_autocorr[ss[5]])
top_autocorr_baised

In [ ]:
ann_dict = dict(sorted(ann_dict.items()))
for ens in top_autocorr_baised:
    i = 0
    fig, axs = plt.subplots(ncols = 3, nrows = 1,figsize = (10,3), facecolor = 'none')
    #[ax.axis('off') for ax in axs.flat]
    for method, ann in ann_dict.items():
        if not method.startswith("ST-FFPE-mIF"):
            continue
        lab = method.replace("ST-FFPE-mIF_","Rep")
        morani = round(ann.uns["moranI"]["I"][ens],2)
        sq.pl.spatial_scatter(
            ann,
            shape = None, 
            cmap = "Reds",
            library_id = 'spatial',
            color = ens,
            ax = axs.flat[i],
            title = f"{lab}\n{ens} Moran's I:{morani}"
        )
        i += 1
    plt.tight_layout()